##### Quick notes on the concept which were not in Deep Learning modules of Machine Learning Nano Degree:

* In text dataset, key terms are more important and we might no have enough dataset describing them. Deep Learning model usually expects many samples to understand the variance of a label. like image of a perticular flower.
*